### 2 modules: array, and nn

In [1]:
import ml.array as ml
import ml.nn as nn

### Numpy-like syntax

In [2]:
a = ml.Array([1,2,3,4])
b = ml.Array([5,6,7,8])

out = (a + b).sqrt().sum()
print(out)

array(11.904297, dtype=float32)


### View computation graphs

In [3]:
out.view_graph()

### Differentiation

In [4]:
a = ml.rand([2,4])
b = ml.rand([4,2])
c = ml.rand([2])
out = (a @ b).sqrt().sum()

# backward 
out.build_backward()
print(a.grad().eval())
print(b.grad().eval())


[[0.44757596 0.39696687 0.50973177 0.3599454 ]
 [0.38673344 0.32849854 0.43344077 0.31179613]]
[[0.7345767  0.39309776]
 [0.9321716  0.50864315]
 [1.1951847  0.64410883]
 [0.72826785 0.36319292]]


In [5]:
out.view_graph()

### Machine learning: Iris dataset

In [6]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def get_data():
    iris = load_iris()
    X, y = iris.data, iris.target
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train = ml.Array(X_train.tolist())
    X_test = ml.Array(X_test.tolist())
    y_train = ml.Array(y_train.tolist())
    y_test = ml.Array(y_test.tolist())
    return X_train, X_test, y_train, y_test

def test_accuracy(W, b):
    X_train, X_test, y_train, y_test = get_data()
    logits_test = X_test.eval() @ W.eval() + (b.eval())
    predictions = np.argmax(logits_test, axis=1)
    accuracy = np.mean((predictions == y_test.eval()))
    print(f'Accuracy: {accuracy.item()}')

In [7]:
def train(lr, iter):
    X_train, X_test, y_train, y_test = get_data()
    one_hot_train = nn.one_hot_encode(y_train, 3)
    
    # fully connected layer
    W = ml.rand([4,3])
    b = ml.rand([3])
    logits = X_train @ W + b

    # built in loss functions
    loss = nn.softmax_cross_entropy_loss(logits, one_hot_train) 
    loss.build_backward()

    # optimizers supported: Adam, SGD
    optim = nn.Adam(lr, [W,b])

    # training loop
    for it in range(iter):
        if (it % (iter//10) == 0): print(f"{it} loss = ", loss.eval().item())
        loss.eval()
        loss.zero_grad()
        loss.set_reeval()
        W.grad().set_reeval()
        b.grad().set_reeval()
        optim.step()
    
    test_accuracy(W,b)
    return loss

lr = 0.01
iter = 100

loss = train(lr,iter)

0 loss =  1.073671817779541
10 loss =  0.34376946091651917
20 loss =  0.2100478857755661
30 loss =  0.15509305894374847
40 loss =  0.12708038091659546
50 loss =  0.11043570190668106
60 loss =  0.10024851560592651
70 loss =  0.09293965995311737
80 loss =  0.0873216837644577
90 loss =  0.08281934261322021
Accuracy: 0.9666666666666667


In [8]:
# view neural network graph
loss.view_graph(view_data=False)